In [401]:
import pandas as pd
import numpy as np

In [402]:
def calculaPorDia(x):    
    #print(x[3]/x[6].days)
    #print(x[3],'',x[7])
    return x[3]/x[7].days


In [431]:
hotels = []
with open('hotels-in-Munich.txt') as f:
    for line in f:
        if len(line)>10:
            hotels.append([str(n.replace('\xa0','').replace('"','').replace('[','').replace(']','')) for n in line.strip().split(';')])
            #print(line)

In [483]:
import datetime
hotelDict = []
for i in hotels:
    #print(i[0][3:5])
    dtIni = datetime.datetime(int(i[0][6:10]), int(i[0][3:5]), int(i[0][0:2]), 0, 0, 0)
    dtEnd = datetime.datetime(int(i[1][6:10]), int(i[1][3:5]), int(i[1][0:2]), 0, 0, 0)
    hotelDict.append({'de':dtIni,'ate':dtEnd,'nome':i[2],'preco':i[3],'nota':i[4],'distancia':i[5],'url':i[6]})

In [497]:
df = pd.DataFrame(hotelDict)
df.drop_duplicates(inplace = True) 
df = df[df['preco']>'-1']
df['preco'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
df['preco'] = pd.to_numeric(df['preco'])
df['dias'] = (df['ate']-df['de']).replace(regex=True,to_replace=r'\D',value=r'')
df["precoPorNoite"] = df.apply(calculaPorDia,axis=1)
display(df.describe())


,preco,dias,precoPorNoite
count,400.000000,400,400.000000
mean,1538.342500,4 days 00:00:00,384.585625
std,406.248002,0 days 00:00:00,101.562000
min,910.000000,4 days 00:00:00,227.500000
25%,1223.000000,4 days 00:00:00,305.750000
50%,1409.000000,4 days 00:00:00,352.250000
75%,1797.000000,4 days 00:00:00,449.250000
max,3522.000000,4 days 00:00:00,880.500000


In [498]:

datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'


#df['diasteste'] = datetime.datetime.strptime(df['ate'],datetimeFormat) - datetime.datetime.strptime(df['de'],datetimeFormat)
df['dias'] = (df['ate']-df['de']).replace(regex=True,to_replace=r'\D',value=r'')
df["nota"] = pd.to_numeric(df["nota"])
df['distanciaMedida'] = pd.np.where(df.distancia.str.contains("km"),"km","m")
df["distanciaNumerica"] = pd.to_numeric(df['distancia'].replace(regex=True,to_replace=r'\D',value=r''))#, downcast='float')
df["distanciaMetros"] = pd.np.where(df.distancia.str.contains("km") & df.distancia.str.contains("."),df.distanciaNumerica*1000,df["distanciaNumerica"])
df["distanciaMetros"] = pd.np.where(df.distanciaMetros > 9000,(df.distanciaNumerica/10),df["distanciaNumerica"])


df["distanciaMetros"] = df["distanciaMetros"].apply(lambda x: x*1000 if x < 100 else x)
#df["precoPorNoite"] = df.apply(calculaPorDia,axis=1)


df = df[df['precoPorNoite']<800]
df = df[df['nota']>7.5]
df = df[df['distanciaMetros']<2000]
df = df[~df["nome"].str.contains('Hostel')]

del df['distanciaMedida']
del df['distanciaNumerica']


df.drop_duplicates(inplace = True) 
display(df.describe())
display(df)




,preco,nota,dias,precoPorNoite,distanciaMetros
count,94.000000,94.000000,94,94.000000,94.000000
mean,1818.170213,8.321277,4 days 00:00:00,454.542553,1124.468085
std,298.383157,0.369211,0 days 00:00:00,74.595789,252.167452
min,1406.000000,7.800000,4 days 00:00:00,351.500000,600.000000
25%,1657.000000,7.925000,4 days 00:00:00,414.250000,1000.000000
50%,1797.000000,8.600000,4 days 00:00:00,449.250000,1100.000000
75%,1896.000000,8.700000,4 days 00:00:00,474.000000,1200.000000
max,2465.000000,8.900000,4 days 00:00:00,616.250000,1900.000000


,de,ate,nome,preco,nota,distancia,url,dias,precoPorNoite,distanciaMetros
6,2020-03-05,2020-03-09,Hotel Eder,2465,8.7,1 km from centre,www.booking.com/hotel/de/eder.en-gb.html?aid=3...,4 days,616.25,1000.0
12,2020-03-05,2020-03-09,Hotel Stachus,1406,7.9,950 m from centre,www.booking.com/hotel/de/stachus-munchen.en-gb...,4 days,351.50,950.0
14,2020-03-05,2020-03-09,Euro Youth Hotel Munich,1797,8.7,1.2 km from centre,www.booking.com/hotel/de/euro-youth.en-gb.html...,4 days,449.25,1200.0
15,2020-03-05,2020-03-09,Novotel Mxc3xcnchen City,1779,8.6,1.4 km from centre,www.booking.com/hotel/de/novotel-munchen.en-gb...,4 days,444.75,1400.0
20,2020-03-05,2020-03-09,Hotel Luitpold,1896,8.0,1.1 km from centre,www.booking.com/hotel/de/luitpoldmuenchen.en-g...,4 days,474.00,1100.0
32,2020-03-05,2020-03-09,Hotel Stachus,1406,7.9,950 m from centre,www.booking.com/hotel/de/stachus-munchen.en-gb...,4 days,351.50,950.0
34,2020-03-05,2020-03-09,Euro Youth Hotel Munich,1797,8.7,1.2 km from centre,www.booking.com/hotel/de/euro-youth.en-gb.html...,4 days,449.25,1200.0
35,2020-03-05,2020-03-09,Novotel Mxc3xcnchen City,1779,8.6,1.4 km from centre,www.booking.com/hotel/de/novotel-munchen.en-gb...,4 days,444.75,1400.0
40,2020-03-05,2020-03-09,Hotel Luitpold,1896,8.0,1.1 km from centre,www.booking.com/hotel/de/luitpoldmuenchen.en-g...,4 days,474.00,1100.0
50,2020-03-05,2020-03-09,City-Hotel Mxc3xcnchen,1853,7.8,1.1 km from centre,www.booking.com/hotel/de/city-munchen.en-gb.ht...,4 days,463.25,1100.0


In [499]:
import plotly.express as px
#fig = px.scatter(df, x="precoPorNoite", y="nota",color="nome")
#fig = px.scatter(df, x="preco", y="nota",color="nome")
#fig.show()

In [500]:
import plotly.express as px
fig = px.scatter(df, x="precoPorNoite", y="nota", size="distanciaMetros", color="nome",hover_name="nome",log_x=True)
fig.show()

In [522]:
df['link'] ='<a href="http://'+df['url']+'">Clique</a>'

df= df.drop_duplicates(subset=['nome', 'preco'])
dfa = df.copy()
del dfa['url']
del dfa['de']
del dfa['ate']
del dfa['dias']
del dfa['distanciaMetros']
dfa = dfa.to_html()
print('de: ',df['de'].max())
print('até: ',df['ate'].max())
#display(dfa)
#display(df.drop_duplicates().sort_values(by=['precoPorNoite'],ascending=False))
from IPython.core.display import display, HTML
display(HTML(dfa))

de:  2020-03-05 00:00:00
até:  2020-03-09 00:00:00


,nome,preco,nota,distancia,precoPorNoite,link
6,Hotel Eder,2465,8.7,1 km from centre,616.25,"<a href=""http://www.booking.com/hotel/de/eder.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b9e793a0c1175bd31ab4942d27b2a032&all_sr_blocks=30946006_96873376_0_1_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=15&highlighted_blocks=30946006_96873376_0_1_0&hpos=7&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=30946006_96873376_0_1_0__54600&srepoch=1578148741&srpvid=955567024616018e&ucfs=1&from=searchresults"">Clique</a>"
12,Hotel Stachus,1406,7.9,950 m from centre,351.50,"<a href=""http://www.booking.com/hotel/de/stachus-munchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b9e793a0c1175bd31ab4942d27b2a032&all_sr_blocks=7440102_91906223_0_2_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=21&highlighted_blocks=7440102_91906223_0_2_0&hpos=13&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=7440102_91906223_0_2_0__31140&srepoch=1578148741&srpvid=955567024616018e&ucfs=1&from=searchresults"">Clique</a>"
14,Euro Youth Hotel Munich,1797,8.7,1.2 km from centre,449.25,"<a href=""http://www.booking.com/hotel/de/euro-youth.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b9e793a0c1175bd31ab4942d27b2a032&all_sr_blocks=7946002_138720173_0_1_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=23&highlighted_blocks=7946002_138720173_0_1_0&hpos=15&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=7946002_138720173_0_1_0__39800&srepoch=1578148741&srpvid=955567024616018e&ucfs=1&from=searchresults"">Clique</a>"
15,Novotel Mxc3xcnchen City,1779,8.6,1.4 km from centre,444.75,"<a href=""http://www.booking.com/hotel/de/novotel-munchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b9e793a0c1175bd31ab4942d27b2a032&all_sr_blocks=6135823_94280284_2_2_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=24&highlighted_blocks=6135823_94280284_2_2_0&hpos=16&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=6135823_94280284_2_2_0__39400&srepoch=1578148741&srpvid=955567024616018e&ucfs=1&from=searchresults"">Clique</a>"
20,Hotel Luitpold,1896,8.0,1.1 km from centre,474.00,"<a href=""http://www.booking.com/hotel/de/luitpoldmuenchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b9e793a0c1175bd31ab4942d27b2a032&all_sr_blocks=6151802_200884162_0_1_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=29&highlighted_blocks=6151802_200884162_0_1_0&hpos=21&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=6151802_200884162_0_1_0__42000&srepoch=1578148741&srpvid=955567024616018e&ucfs=1&from=searchresults"">Clique</a>"
50,City-Hotel Mxc3xcnchen,1853,7.8,1.1 km from centre,463.25,"<a href=""http://www.booking.com/hotel/de/city-munchen.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID&sid=b69b2d1a4e5dbbd249d68399fdab0402&all_sr_blocks=6612002_97883604_0_2_0&checkin=2020-03-05&checkout=2020-03-09&dest_id=-1829149&dest_type=city&group_adults=2&group_children=0&hapos=38&highlighted_blocks=6612002_97883604_0_2_0&hpos=22&nflt=roomfacility%3D38%3B&no_rooms=1&sr_order=popularity&sr_pri_blocks=6612002_97883604_0_2_0__41041&srepoch=1578148741&srpvid=3e686702896f00c7&ucfs=1&from=searchresults"">Clique</a>"
53,Hotel MIO y AMANO,1624,8.6,600 m from centre,406.00,"<a href=""http://www.booking.com/hotel/de/mio-by-amano.en-gb.html?aid=304142&label=gen173nr-1BCAEoggI46AdIM1gEaJMBiAEBmAEJuAEXyAEM2AEB6AEBiAIBqAID